<a href="https://colab.research.google.com/github/dlwldms5/jeonbotdae/blob/main/%EC%A0%84%EB%B4%87%EB%8C%80%EA%B3%BC%EC%A0%9C3_%EC%9D%B4%EC%A7%80%EC%9D%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
from math import atan2, degrees, sqrt
from google.colab import files

uploaded = files.upload()
video_path = next(iter(uploaded))

class Vehicle:
    def __init__(self, object_id, centroid):
        self.id = object_id
        self.centroids = [centroid]
        self.speeds = []
        self.directions = []

    def update(self, centroid, fps, pixel_to_meter=0.05):
        self.centroids.append(centroid)
        if len(self.centroids) >= 2:
            dx = self.centroids[-1][0] - self.centroids[-2][0]
            dy = self.centroids[-1][1] - self.centroids[-2][1]
            distance_pixels = sqrt(dx ** 2 + dy ** 2)
            distance_meters = distance_pixels * pixel_to_meter
            speed = distance_meters * fps
            direction = degrees(atan2(-dy, dx)) % 360
            self.speeds.append(speed)
            self.directions.append(direction)

class CentroidTracker:
    def __init__(self, max_disappeared=30, max_distance=50):
        self.next_object_id = 0
        self.objects = {}
        self.disappeared = {}
        self.vehicles = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance

    def register(self, centroid):
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.vehicles[self.next_object_id] = Vehicle(self.next_object_id, centroid)
        self.next_object_id += 1

    def deregister(self, object_id):
        del self.objects[object_id]
        del self.disappeared[object_id]
        del self.vehicles[object_id]

    def update(self, detections, fps):
        if len(detections) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        input_centroids = np.array(
            [(int((x1 + x2) / 2), int((y1 + y2) / 2)) for (x1, y1, x2, y2) in detections]
        )

        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)
        else:
            object_ids = list(self.objects.keys())
            object_centroids = list(self.objects.values())

            D = np.linalg.norm(
                np.array(object_centroids)[:, None] - input_centroids[None, :],
                axis=2
            )

            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            used_rows, used_cols = set(), set()

            for row, col in zip(rows, cols):
                if row in used_rows or col in used_cols:
                    continue
                if D[row, col] > self.max_distance:
                    continue

                object_id = object_ids[row]
                self.objects[object_id] = input_centroids[col]
                self.disappeared[object_id] = 0
                self.vehicles[object_id].update(input_centroids[col], fps)

                used_rows.add(row)
                used_cols.add(col)

            unused_cols = set(range(len(input_centroids))) - used_cols
            for col in unused_cols:
                self.register(input_centroids[col])

            unused_rows = set(range(len(object_ids))) - used_rows
            for row in unused_rows:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

        return self.objects

    def draw(self, frame):
        for object_id, centroid in self.objects.items():
            x, y = int(centroid[0]), int(centroid[1])
            vehicle = self.vehicles[object_id]
            label = f"ID:{object_id}"
            if vehicle.speeds:
                label += f" V:{vehicle.speeds[-1]:.2f}m/s D:{vehicle.directions[-1]:.1f}°"

            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(frame, label, (x + 10, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

def dummy_detector(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, th = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h > 500:
            detections.append((x, y, x + w, y + h))
    return detections

cap = cv2.VideoCapture(video_path)
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_final.mp4', fourcc, fps, (width, height))

tracker = CentroidTracker()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detections = dummy_detector(frame)
    tracker.update(detections, fps)
    tracker.draw(frame)
    out.write(frame)

cap.release()
out.release()

files.download('output_final.mp4')


Saving 자동차영상2.mp4 to 자동차영상2.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>